In [59]:
import sys
sys.path.append("../../")

import torch
import torchvision.utils
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import optim
from torch.utils.data import DataLoader

import numpy as np

import src.neuralNetworksArch.BasicSiamese as bSiamese
import src.utils.Visual as vis
import src.utils.DatasetLoader as dsetLoader
import src.utils.LossFunction as lossFunc

from src.config.Param import *

In [60]:
MODEL_PATH = '../../models/full-contrastive-basicSiamese-100epoch.pth'
DATATEST_PATH = '../../dataset/cuhk02/datatest.csv'
IMAGES_PATH = '../../dataset/cuhk02/images/full/'

In [61]:
trans = transforms.Compose([transforms.ToTensor()])
test_dataset = dsetLoader.ContrastiveDataset(csv_path=DATATEST_PATH, images_path=IMAGES_PATH, transform=trans)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [62]:
model  = bSiamese.BasicSiameseNetwork()
model.load_state_dict(torch.load(MODEL_PATH, map_location=Param.device))
model.eval()

BasicSiameseNetwork(
  (cnn1): Sequential(
    (0): ReflectionPad2d((1, 1, 1, 1))
    (1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
    (2): ReLU(inplace=True)
    (3): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReflectionPad2d((1, 1, 1, 1))
    (5): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU(inplace=True)
    (7): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): ReflectionPad2d((1, 1, 1, 1))
    (10): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (11): ReLU(inplace=True)
    (12): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(30, 80))
  (fc1): Sequential(
    (0): Linear(in_features=38400, out_features=1000, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=1000, out_features=500, bias=T

In [63]:
dataiter = iter(test_dataloader)
with torch.no_grad():
    for i in range(len(test_dataloader)):
        img1, img2, label = next(dataiter)
        concatenated = torch.cat((img1, img2),0)

        img1 = img1.to(Param.device)
        img2 = img2.to(Param.device)
        label = label.to(Param.device)
        
        out1, out2 = model(img1, img2)
        euclidean_distance = F.pairwise_distance(out1, out2)
        d = float(euclidean_distance.item())
        d = abs((1 / (1 + d)) - 1)
        
        print('Dissimilarity', d)
        vis.imshow(torchvision.utils.make_grid(concatenated))

StopIteration: 